# Setup

In [1]:
# was needed to download files from repo

# Download TorchVision repo to use some files from
# references/detection
# !git clone https://github.com/pytorch/vision.git
# !cd vision
# !git checkout v0.3.0

# !cp references/detection/utils.py ../
# !cp references/detection/transforms.py ../
# !cp references/detection/coco_eval.py ../
# !cp references/detection/engine.py ../
# !cp references/detection/coco_utils.py ../

# Imports

In [2]:
import fiftyone as fo
import fiftyone.utils.coco as fouc
from fiftyone.core.labels import Detection
from PIL import Image

import torch
import torch.nn as nn
import torch.utils
import torch.utils.data

import torchvision
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights
from torchvision.models.detection.mask_rcnn import maskrcnn_resnet50_fpn, MaskRCNNPredictor
from torchvision.models.detection import MaskRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

import matplotlib.pyplot as plt
import numpy as np

import os
import importlib
from pathlib import Path

import utils
import train
from train import train_one_epoch, evaluate
import transforms as T
import ENEE439.Capstone.models.DatasetLoaders as DatasetLoaders
importlib.reload(DatasetLoaders)
from ENEE439.Capstone.models.DatasetLoaders import HRSIDSegmentationDataset


In [3]:
importlib.reload(utils)
importlib.reload(T)
importlib.reload(train)

<module 'train' from '/home/k3vinli/ENEE439/Capstone/models/mask_rcnn/train.py'>

# loading data into fiftyone

In [4]:
p = Path()
top_dir = p.absolute().parents[1]
top_dir

PosixPath('/home/k3vinli/ENEE439/Capstone')

In [5]:
# Loading Dataset
name = "HRSID"
if name in fo.list_datasets():
    dataset_traintest = fo.load_dataset(name)
else:
    dataset_dir = top_dir / "Datasets" / "HRSID"
    # The type of the dataset being imported
    dataset_type = fo.types.COCODetectionDataset

    dataset_traintest = fo.Dataset.from_dir(
        dataset_dir=dataset_dir,
        dataset_type=dataset_type,
        name=name,
    )

  33% |████-----------| 1861/5604 [8.0s elapsed, 16.6s remaining, 212.8 samples/s]    

In [ ]:
# dataset_traintest.delete()

In [ ]:
name = "HRSID_train"
if name in fo.list_datasets():
    dataset_train = fo.load_dataset(name)
else:
    dataset_dir = top_dir / "Datasets" / "HRSID"
    label_path = top_dir / "Datasets"/ "HRSID" / "annotations" / "train2017.json"
    # The type of the dataset being imported
    dataset_type = fo.types.COCODetectionDataset

    dataset_train = fo.Dataset.from_dir(
        dataset_dir=dataset_dir,
        dataset_type=dataset_type,
        name=name,
        labels_path=label_path
    )

 100% |███████████████| 3642/3642 [20.7s elapsed, 0s remaining, 136.2 samples/s]      


In [ ]:
# dataset_train.delete()

In [ ]:
name = "HRSID_test"
if name in fo.list_datasets():
    dataset_test = fo.load_dataset(name)
else:
    dataset_dir = top_dir / "Datasets" / "HRSID"
    label_path = top_dir / "Datasets"/ "HRSID" / "annotations" / "test2017.json"
    # The type of the dataset being imported
    dataset_type = fo.types.COCODetectionDataset

    dataset_test = fo.Dataset.from_dir(
        dataset_dir=dataset_dir,
        dataset_type=dataset_type,
        name=name,
        labels_path=label_path
    )

 100% |███████████████| 1961/1961 [11.0s elapsed, 0s remaining, 104.8 samples/s]     


In [ ]:
dataset_train.compute_metadata()
dataset_test.compute_metadata()

# explore fiftyone useage

In [ ]:
paths = dataset_train.values("filepath")
sample = dataset_train[paths[1]]
sample.metadata

<ImageMetadata: {
    'size_bytes': None,
    'mime_type': None,
    'width': 800,
    'height': 800,
    'num_channels': None,
}>

In [ ]:
#sample

In [ ]:
img = Image.open(paths[1]).convert("RGB")
detections = sample["detections"].detections
segmentations = sample["segmentations"].detections

In [ ]:
classes = dataset_train.distinct("%s.detections.label" % "segmentations")

In [ ]:
# segmentations

In [ ]:
labels_map_rev = {c: i for i, c in enumerate(classes)}
labels_map_rev


{'ship': 0}

In [ ]:
for det in segmentations:
    category_id=labels_map_rev[det.label]
    coco_obj=fouc.COCOObject.from_label(det, sample.metadata, category_id=category_id)
    x,y,w,h=coco_obj.bbox

In [ ]:
# [coco_obj.segmentation], dtype=torch.float

In [ ]:
sample = dataset_test.first()
frame_size = (sample.metadata["width"], sample.metadata["height"])
detection = sample["segmentations"]["detections"][0]

segmentation = detection.to_segmentation(frame_size=frame_size)
full_img_mask = segmentation.mask
print(type(detection))
print("frame size", frame_size)
print("detection:", detection)
print("segmentation:", segmentation)
print("full img", full_img_mask.max())

<class 'fiftyone.core.labels.Detection'>
frame size (800, 800)
detection: <Detection: {
    'id': '645524015548b8a34c22caa0',
    'attributes': {},
    'tags': [],
    'label': 'ship',
    'bounding_box': [0.29625, 0.09875, 0.0525, 0.01],
    'mask': array([[False, False, False, False, False, False, False, False, False,
            False, False, False, False, False, False, False,  True,  True,
             True,  True,  True,  True,  True,  True,  True,  True,  True,
             True,  True,  True,  True,  True,  True,  True,  True,  True,
             True,  True, False, False, False, False],
           [False,  True,  True,  True,  True,  True,  True,  True,  True,
             True,  True,  True,  True,  True,  True,  True,  True,  True,
             True,  True,  True,  True,  True,  True,  True,  True,  True,
             True,  True,  True,  True,  True,  True,  True,  True,  True,
             True,  True,  True,  True,  True, False],
           [ True,  True,  True,  True,  Tr

In [ ]:
session = fo.launch_app()

# Training ResNet Model

In [ ]:
def get_model(num_classes, pretrained=True):
    if pretrained:
        weights = MaskRCNN_ResNet50_FPN_Weights.DEFAULT
    else:
        weights = None
    model = maskrcnn_resnet50_fpn(weights=weights)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [ ]:
def get_transform():
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    
    return T.Compose(transforms)

In [ ]:
def do_training(model, torch_dataset, torch_dataset_test, num_epochs=4):
    data_loader = torch.utils.data.DataLoader(
        torch_dataset, batch_size=4, shuffle=False, num_workers=0,
        collate_fn=utils.collate_fn
    )
    data_loader_test = torch.utils.data.DataLoader(
        torch_dataset_test, batch_size=2, shuffle=False, num_workers=0,
        collate_fn=utils.collate_fn
    )

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using {device}")
    model.to(device)

    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.00001,
                                momentum=0.3, weight_decay=0.0005)

    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                    step_size=5,
                                                    gamma=0.1)
    for epoch in range(num_epochs):
        train_one_epoch(model=model,
                        optimizer=optimizer, 
                        data_loader=data_loader,
                        device=device, 
                        epoch=epoch, 
                        print_freq=1)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        lr_scheduler.step()
        evaluate(model=model, data_loader=data_loader_test, device=device)

In [ ]:
HRSID_train = HRSIDSegmentationDataset(dataset_train, transforms=get_transform())
HRSID_test = HRSIDSegmentationDataset(dataset_test, transforms=get_transform())

In [ ]:
x = np.array([0,0,0,255,255,0,0,0])
np.clip(x, 0, 1)

array([0, 0, 0, 1, 1, 0, 0, 0])

In [ ]:
model = get_model(2)

In [ ]:
do_training(model, HRSID_train, HRSID_test, num_epochs=20)


Using cuda
Epoch: [0]  [  0/911]  eta: 0:35:35  lr: 0.000000  loss: 437.1587 (437.1587)  loss_classifier: 0.9016 (0.9016)  loss_box_reg: 0.0878 (0.0878)  loss_mask: 433.9677 (433.9677)  loss_objectness: 1.9407 (1.9407)  loss_rpn_box_reg: 0.2609 (0.2609)  time: 2.3440  data: 1.8416  max mem: 5022
Epoch: [0]  [ 10/911]  eta: 0:27:19  lr: 0.000000  loss: 308.5089 (328.1227)  loss_classifier: 0.8862 (0.8869)  loss_box_reg: 0.1620 (0.1496)  loss_mask: 307.2666 (326.4211)  loss_objectness: 0.2107 (0.5879)  loss_rpn_box_reg: 0.0288 (0.0773)  time: 1.8194  data: 0.8877  max mem: 5205
Epoch: [0]  [ 20/911]  eta: 0:28:34  lr: 0.000000  loss: 265.7952 (275.9522)  loss_classifier: 0.8922 (0.8959)  loss_box_reg: 0.1091 (0.1339)  loss_mask: 264.5305 (274.1040)  loss_objectness: 0.1410 (0.7349)  loss_rpn_box_reg: 0.0262 (0.0834)  time: 1.9030  data: 0.9101  max mem: 5234
Epoch: [0]  [ 30/911]  eta: 0:27:16  lr: 0.000000  loss: 160.6520 (188.3371)  loss_classifier: 0.8988 (0.8955)  loss_box_reg: 0.114

SystemExit: 1

In [ ]:
%tb

SystemExit: 1

In [ ]:
model=None
torch.cuda.empty_cache()
